In [40]:
import time, sys
import json
import random
from PIL import Image
import numpy as np
from tinygrad.tensor import Tensor
from tinygrad.nn import optim, BatchNorm2d
import tinygrad.nn as nn
from tinygrad.helpers import flatten
from tinygrad.nn.optim import SGD, Adam
from sklearn.datasets import fetch_openml
from tinygrad.state import safe_save, safe_load, get_state_dict, load_state_dict, torch_load

In [41]:
state_dict = {}
file = open("weights_bias/monkey_weight_bias_model.txt", "r")

conv1_weights = file.readline()
conv1_bias = file.readline()

conv2_weights = file.readline()
conv2_bias = file.readline()

conv3_weights = file.readline()
conv3_bias = file.readline()

fc1_weights = file.readline()
fc1_bias = file.readline()

fc2_weights = file.readline()
fc2_bias = file.readline()

fc3_weights = file.readline()
fc3_bias = file.readline()

In [42]:
def set_weight_bias_format(data, shape):
    data = data.split(" ")
    data = list(map(float, data))
    return np.array(data).reshape(shape)

In [43]:
conv1_weights = set_weight_bias_format(conv1_weights, (16, 3, 3, 3))
conv1_bias = set_weight_bias_format(conv1_bias, 16)

conv2_weights = set_weight_bias_format(conv2_weights, (32, 16, 3, 3))
conv2_bias = set_weight_bias_format(conv2_bias, (32))

conv3_weights = set_weight_bias_format(conv3_weights, (64, 32, 3, 3))
conv3_bias = set_weight_bias_format(conv3_bias, (64))

fc1_weights = set_weight_bias_format(fc1_weights, (100, 1024))
fc1_bias = set_weight_bias_format(fc1_bias, (100))

fc2_weights = set_weight_bias_format(fc2_weights, (50, 100))
fc2_bias = set_weight_bias_format(fc2_bias, (50))

fc3_weights = set_weight_bias_format(fc3_weights, (10, 50))
fc3_bias = set_weight_bias_format(fc3_bias, (10))

In [44]:
state_dict["conv1.weight"] = Tensor(conv1_weights.astype(np.float32))
state_dict["conv1.bias"] = Tensor(conv1_bias.astype(np.float32))

state_dict["conv2.weight"] = Tensor(conv2_weights.astype(np.float32))
state_dict["conv2.bias"] = Tensor(conv2_bias.astype(np.float32))

state_dict["conv3.weight"] = Tensor(conv3_weights.astype(np.float32))
state_dict["conv3.bias"] = Tensor(conv3_bias.astype(np.float32))

state_dict["fc1.weight"] = Tensor(fc1_weights.astype(np.float32))
state_dict["fc1.bias"] = Tensor(fc1_bias.astype(np.float32))

state_dict["fc2.weight"] = Tensor(fc2_weights.astype(np.float32))
state_dict["fc2.bias"] = Tensor(fc2_bias.astype(np.float32))

state_dict["fc3.weight"] = Tensor(fc3_weights.astype(np.float32))
state_dict["fc3.bias"] = Tensor(fc3_bias.astype(np.float32))

In [45]:
class MonkeyTinyNet:
    def __init__(self):
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        
        self.fc1 = nn.Linear(in_features=64*4*4, out_features=100)
        self.fc2 = nn.Linear(in_features=100, out_features=50)
        self.fc3 = nn.Linear(in_features=50, out_features=10)

    def forward(self, x):
        x = x.reshape((x.shape[0], 3, 128, 128))

        x = self.conv1(x)
        x = x.relu()
        x = x.max_pool2d(2)  
        # extract_feature_map(x)

        x = self.conv2(x)
        x = x.relu()
        x = x.max_pool2d(2)        

        x = self.conv3(x)
        x = x.relu()
        x = x.max_pool2d(2)         

        x = x.reshape(x.shape[0], -1)

        x = self.fc1(x) 
        x = x.relu()

        x = self.fc2(x) 
        x = x.relu()

        x = self.fc3(x)
        return x.log_softmax()
        

checkpoints = {
    'conv1.weight': state_dict['conv1.weight'],
    'conv1.bias': state_dict['conv1.bias'],
    'conv2.weight': state_dict['conv2.weight'],
    'conv2.bias': state_dict['conv2.bias'],
    'conv3.weight': state_dict['conv3.weight'],
    'conv3.bias': state_dict['conv3.bias'],
    'fc1.weight': state_dict['fc1.weight'],
    'fc1.bias': state_dict['fc1.bias'],
    'fc2.weight': state_dict['fc2.weight'],
    'fc2.bias': state_dict['fc2.bias'],
    'fc3.weight': state_dict['fc3.weight'],
    'fc3.bias': state_dict['fc3.bias']
}

model = MonkeyTinyNet()
load_state_dict(model, checkpoints)

ram used:  0.00 GB, fc3.bias                                          : 100%|██████████| 12/12 [00:00<00:00, 554.15it/s]

loaded weights in 24.38 ms, 0.00 GB loaded at 0.08 GB/s


In [46]:
import torchvision.transforms as transforms

def pytorch_preprocessing(image):  
    transform = transforms.Compose([
        transforms.Resize((128, 128)),  # Apply the necessary transformations
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    image = transform(image)
    image = image.unsqueeze(0) 
    return Tensor(image.numpy())

In [61]:
import time

monkey_classes = ['Emperor Tamarin', 'Gray Langur', 'Hamadryas Baboon', 'Proboscis Monkey', 'Vervet Monkey', 
                    'Golden Monkey', 'Mandril', 'Bald Uakari', 'White Faced Saki', 'Red Howler']

def inference_model(image):
    image = pytorch_preprocessing(image)
    output_tensor = model.forward(image) 

    predicted_class= np.argmax(output_tensor.numpy(), axis=-1)
    return monkey_classes[predicted_class[0]]

In [48]:
image = Image.open('test_img/White_Faced_Saki.jpg')
pred_class =  inference_model(image)
print("Predicted class:", pred_class)

Predicted class: White Faced Saki
Inference Time Taken:  0.014865398406982422


In [49]:
image = Image.open('test_img/Hamadryas_Baboon.jpg')
pred_class = inference_model(image)
print("Predicted class:", pred_class)

Predicted class: Hamadryas Baboon
Inference Time Taken:  0.009249448776245117


In [64]:
test_folder = "monkey_test_data/"
each_folder = os.listdir(test_folder)

acc = 0
total_imgs = 0

start_time = time.time()
for folder in each_folder:
    folder_path = os.path.join(test_folder, folder)
    for img_name in os.listdir(folder_path):
        image = Image.open(f"{folder_path}/{img_name}")
        pred_class = inference_model(image)
        if pred_class == folder:
            acc+=1

        total_imgs+=1

print("Inference Time Taken: ", time.time()-start_time)
print(f"Accuracy over {total_imgs} Test Images: {acc/total_imgs}")

Inference Time Taken:  16.52835464477539
Accuracy over 1306 Test Images: 0.7366003062787136
